In [4]:
import requests
import pandas as pd

# Получаем список всех пар
url = "https://api.kraken.com/0/public/AssetPairs"
resp = requests.get(url).json()
usdc_pairs = []
for key, val in resp['result'].items():
    if val.get('wsname') and '/USDC' in val['wsname'] and '.d' not in key:
        usdc_pairs.append({'pair': key, 'wsname': val['wsname']})

# Получаем тикер и объём по каждой паре
good_pairs = []
for p in usdc_pairs:
    ticker_url = f"https://api.kraken.com/0/public/Ticker?pair={p['pair']}"
    resp = requests.get(ticker_url).json()
    result = list(resp['result'].values())[0]
    bid = float(result['b'][0])
    ask = float(result['a'][0])
    spread = (ask - bid) / ((ask + bid) / 2) * 100  # в %
    volume_24h = float(result['v'][1])
    if spread < 0.2 and volume_24h > 100000:  # спред меньше 0.2%, объем больше $100K
        good_pairs.append({
            'pair': p['wsname'],
            'spread_%': spread,
            'volume_24h': volume_24h
        })

# Выводим отобранные пары
df = pd.DataFrame(good_pairs).sort_values('volume_24h', ascending=False)
print("Пары с низким спредом и высокой ликвидностью:")
print(df)


Пары с низким спредом и высокой ликвидностью:
       pair  spread_%    volume_24h
0  ADA/USDC  0.181168  3.766546e+06
1  XRP/USDC  0.000410  6.332365e+05
